In [1]:
import torch 
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[0.0457],
        [0.2397]], grad_fn=<AddmmBackward0>)

In [2]:
#参数访问,通过Sequential类定义模型时,通过索引来访问模型的任意层,这就像模型是一个列表一样,每层的参数都在属性中
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1326,  0.3335,  0.3239, -0.3307,  0.0770,  0.1649,  0.1977, -0.3249]])), ('bias', tensor([0.0694]))])


>这个全连接层包含两个参数,权重和偏置都是单精度浮点型(float32)

In [3]:
#目标参数,每个参数都表示为参数类的一个实例,要对参数执行任何操作,首先我们就要访问底层的数值
#从第二个全连接层提取偏置,提取后返回一个参数类实例
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0694], requires_grad=True)
tensor([0.0694])


In [4]:
#由于还没有调用反向传播,所以参数的梯度处于初始状态
net[2].weight.grad==None

True

In [6]:
#一次性访问所有参数
#当我们需要对所有参数执行操作时逐个访问可能会很麻烦
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

#提供另一种访问网络参数的方式
#state_dict()返回网络的参数字典
net.state_dict()['2.bias'].data

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([0.0694])

In [7]:
#从嵌套块收集参数
#定义一个生成块的函数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net=nn.Sequential()
    for i in range(4):
        #在这里嵌套
        net.add_module(f'block{i}',block1())
    return net
rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.2561],
        [-0.2561]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
rgnet[0][1][0].bias.data

tensor([-0.1694,  0.2806,  0.3057,  0.0724, -0.4863,  0.0745, -0.4041,  0.2573])

#参数初始化
>知道了如何访问参数之后,现在看看如何正确地初始化参数,深度学习框架提供默认随机初始化,也允许我们创建自定义初始化方法,满足我们通过其他规则实现初始化权重
默认情况下,PyTorch会根据一个范围均匀地初始化权重和偏置矩阵,这个范围是根据输入和输出维度计算出的,PyTorch的nn.init模块提供了多种预置初始化方法

In [10]:
#内置初始化
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0015, -0.0052, -0.0028, -0.0029]), tensor(0.))

In [11]:
#将所有参数初始化为给定的常数,初始化为1
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

In [12]:
#我们对某些块应用不同的初始化方法,我们使用Xavier初始化方法初始化第一个神经网络层,然后将三个神经网络层初始化为常量42
def init_xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5974,  0.0464,  0.6088,  0.2930])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [15]:
#自定义初始化,有时深度学习框架没有提供我们需要的初始化方法
def my_init(m):
    if type(m)==nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs()>=5
net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.9992,  8.1463, -7.8361,  8.9164],
        [ 7.4603,  0.0000, -5.3664,  0.0000]], grad_fn=<SliceBackward0>)

In [17]:
#参数绑定
#定义一个稠密层,使用他的参数来设置另一个层的参数
#给共享层一个名称,以便引用它的参数
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))
net(X)

#检查参数是否相同
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
#确保他们是同一个对象,而不是有相同的值
print(net[2].weight.data[0]==net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


>如果我们改变其中一个参数,另一个参数也会发生改变,第三个和第五个神经网络层的参数是绑定的,它们不仅值相等,而且由相同的张量表示


### 延后初始化
>到目前为止,我们忽略了建立网络时需要做的以下这些事情:
>* 我们定义了网络架构,但没有指定输入维度
>* 我们添加层时没有指定前一层的输出维度
>* 我们在初始化参数时,甚至没有足够的信息来确定模型应该包含多少参数
>深度学习框架无法判断网络的输入维度是什么,这里的诀窍是框架的延后初始化,即直到数据第一次通过模型传递时,框架才会动态地推断出每个层的大小
>在以后,当使用卷积神经网络时,由于输入维度(图像的分辨率)将影响后续层的维数,有了该技术将更加方便
